# Bibliotecas

In [1]:
import pandas as pd # Biblioteca necessária para a manipulação de dataframes
import requests # Biblioteca necessária para requisições das API's que serão utilizadas
import json # Biblioteca necessária para manipulação de arquivos .json
import openai # Biblioteca para a utilização da API da OpenAI

# URL da API

In [2]:
# Repositório da API : https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

# EXTRACT

Extração das informações presentes no arquivo SDW2023.csv

In [3]:
df = pd.read_csv('SDW2023.csv')
users_ids = df['UserID'].to_list()

Aquisição dos dados dos cliente via API

In [4]:
def get_user(id):
    response = requests.get(f'{sdw2023_api_url}/users/{id}')

    return response.json() if response.status_code == 200 else None

users = [user for id in users_ids if (user := get_user(id)) is not None]
# print(json.dumps(users, indent= 2))

# TRANSFORM

Para iniciar a etapa de transformação é necessário antes instalar e preparar o ambiente para o mesmo utilizar a biblioteca de OpenAI, Para isso basta seguir os seguinte passos:

1. Instale a biblioteca pelo comando "!pip install openai" ;

2. Crie uma conta na OpenAI. (https://openai.com/) ;

3. Crie uma APIkey para a sua aplicação (https://platform.openai.com/account/api-keys). Nesse exercicio a APIkey vai aparecer protegida, mas será indicado quando você pode substituir a mesma por sua APIkey.

WARNING: Durante a elaboração desse desafio foi verificado que o chat GPT so permite TRÊS consultas por minuto (No plano utilizado para o estudo). Portanto para fins ditaticos o arquivo .csv só possui o ID de 3 usuarios.

In [5]:
## Obtenção da APIkey
with open('ignore/token_openai.txt', 'r') as arquivo:
    openai_api_key = arquivo.read()

openai.api_key = openai_api_key # Informando para a API o token de acesso

Com tudo configurado podemos iniciar a geração de mensagens para o cliente. Isso será efetuado com a ajuda da documentação disponivel no link: https://platform.openai.com/docs/api-reference/chat/create?lang=python

In [6]:
def generate_ai_news(user):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system", 
                "content": "Você é um especialista em marketing bancário."
            },
            {
                "role": "user", 
                "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)"
            }
        ]
    )

    return completion.choices[0].message.content

for user in users:

    news = generate_ai_news(user)
    user['news'].append({
        "description": news
    })
    print(news)

Invista agora e garanta seu futuro!
Invista hoje e garanta seu futuro financeiro! #InvestimentosImportantes
"Invista no seu futuro: faça seus recursos trabalharem por você!"


# LOAD

Agora podemos iniciar a atualização das mensagens para a API.

In [16]:
def update_user(user):
    response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)

    return True if response.status_code == 200 else False

for user in users:

    sucess = update_user(user)

    print(f"User {user['name']} updated? {sucess}!")

User Devweekerson updated? False!
User anguile updated? True!
User Pyterson updated? True!
